In [47]:
import numpy as np
from scipy.special import erfc

# Q-function using erfc for Gaussian distribution
def Q(x):
    return 0.5 * erfc(x / np.sqrt(2))

# Calculate gamma values for tag and message based on alpha, signal power P, and noise power sigma_n
def calculate_gamma(alpha, P, sigma_n):
    gamma_t = (alpha * P) / ((1 - alpha) * P + sigma_n**2)
    gamma_m = ((1 - alpha) * P) / (alpha * P + sigma_n**2)
    return gamma_t, gamma_m

# Calculate Bit Error Rate (BER) p_t and p_m based on gamma_t and gamma_m
def calculate_p(gamma):
    return Q(np.sqrt(2 * gamma))

# Define C' and C'' based on p_t and p_m using the channel capacity formula
def calculate_C(p):
    return 1 + p * np.log2(p) + (1 - p) * np.log2(1 - p) if p > 0 else 1  # Handling p=0 for log stability

# Calculating Pe for tag and message lengths based on the provided formulas
def calculate_Pe(R, p, C, length, x=1):
    if x == 1:
        return Q(np.sqrt(length / (p * (1 - p))) * ((C - R) / np.log2((1 - p) / p)))
    else:
        return Q(np.sqrt(x * length / (p * (1 - p))) * ((C - (t/x*m)*R) / np.log2((1 - p) / p)))

# Calculate AER based on the provided formula
def calculate_AER(Pe_m_prime, Pe_t_prime, Pe_t, x, y):
    term1 = Pe_m_prime
    term2 = (1 - (1 - Pe_m_prime)**(y-1) * (1 - Pe_t_prime))
    term3 = (1 - (1 - Pe_m_prime)**(x-1) * (1 - Pe_t))
    AER = term1 + term2 * term3 * (1 - Pe_m_prime)
    return AER

# Arbitrary values for parameters
P = 2  # Signal power
sigma_n = 1  # Noise power
alpha_scheme_1 = 0  # Alpha for scheme 1
alpha_scheme_2 = 0.1  # Alpha for scheme 2
R = .5  # Transmission rate
t = 128  # Length of the tag
m = 128  # Length of the message
x = 6  # Arbitrary parameter
y = 3  # Arbitrary parameter

# Scheme 1 calculations
gamma_t_1, gamma_m_1 = calculate_gamma(alpha_scheme_1, P, sigma_n)
p_t_1 = calculate_p(gamma_m_1)
p_m_1 = calculate_p(gamma_m_1)
C_1 = calculate_C(p_t_1)
print("C1:", C_1)
Pe_t_1 = calculate_Pe(R, p_t_1, C_1, t, x=1)
Pe_m_1 = calculate_Pe(R, p_m_1, C_1, m)
print("Pe_t:", Pe_t_1)
print("Pe_m:", Pe_m_1)
AER_scheme_1 = calculate_AER(Pe_m_1, Pe_t_1, Pe_t_1, x, y)

# Scheme 2 calculations
gamma_t_2, gamma_m_2 = calculate_gamma(alpha_scheme_2, P, sigma_n)
p_t_2 = calculate_p(gamma_t_2)
p_m_2 = calculate_p(gamma_m_2)
C_prime_2 = calculate_C(p_t_2)
C_double_prime_2 = calculate_C(p_m_2)
print("C_prime_2:", C_prime_2)
print("C_double_prime_2:", C_double_prime_2)
Pe_t_prime_2 = calculate_Pe(R, p_t_2, C_prime_2, t, x=x)
Pe_m_prime_2 = calculate_Pe(R, p_m_2, C_double_prime_2, m)
AER_scheme_2 = calculate_AER(Pe_m_prime_2, Pe_t_prime_2, Pe_t_1, x, y)

AER_scheme_1, AER_scheme_2

C1: 0.8433849138748961
Pe_t: 7.816738616943528e-07
Pe_m: 7.816738616943528e-07
C_prime_2: 0.06351874609186847
C_double_prime_2: 0.7502733767949537


(7.816848599081365e-07, 0.005183211769872745)